In [1]:
import time
from tqdm import tqdm
import json

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("csv_files/annotated_corrected_with_contexts.csv")

In [4]:
df.columns = ['submissionId', 'submissionURL', 'submissionTitle', 'context1', 'context2', 'utterance','isTextSarcastic?', 'isImageSarcastic?', 'isTogetherSarcastic?']

In [5]:
df.head()

,submissionId,submissionURL,submissionTitle,context1,context2,utterance,isTextSarcastic?,isImageSarcastic?,isTogetherSarcastic?
0,1dncran,https://i.redd.it/1xvccvq6oi8d1.jpeg,the clouds did a thing this morning...,looks like someone laid out some rails of blow,all the deities got together are they are havi...,colombian clouds,1,0,1
1,c9tdn7,https://i.imgur.com/6KzNep5.jpg,looks like i won't be listening to my new viny...,it’s even crammed in there so hard your mailbo...,is there's a way to ask for a refund from the ...,how can an assigned delivery person not know t...,0,0,1
2,kctb6e,https://i.redd.it/9knprglwx3561.jpg,sounded way better in my head,i hope we get our 9-month cupcake in a cup.,bruh wall-e's my fav pixar movie! take my upvo...,i remember when it was just supposed to be a m...,0,1,1
3,oagyyo,https://i.redd.it/dm6sx4nqn9871.jpg,was just trying to help the driver.,orders $10 in food. tries to leave $5.50 tip. ...,more that?,maybe it prevents accidentally doing the wrong...,0,0,1
4,faw3bs,https://i.imgur.com/gwVDXvB.jpg,wow,"holy shit, i've seen a lot of these but that i...",the really boring dorian grey,step 1: dress up as subject.\n\nstep 2: walk u...,1,1,1


In [6]:
columns_to_keep = ['submissionId', 'submissionTitle', 'context1', 'context2', 'utterance','isTextSarcastic?']
df = df[columns_to_keep]

In [7]:
df.head(10)

,submissionId,submissionTitle,context1,context2,utterance,isTextSarcastic?
0,1dncran,the clouds did a thing this morning...,looks like someone laid out some rails of blow,all the deities got together are they are havi...,colombian clouds,1
1,c9tdn7,looks like i won't be listening to my new viny...,it’s even crammed in there so hard your mailbo...,is there's a way to ask for a refund from the ...,how can an assigned delivery person not know t...,0
2,kctb6e,sounded way better in my head,i hope we get our 9-month cupcake in a cup.,bruh wall-e's my fav pixar movie! take my upvo...,i remember when it was just supposed to be a m...,0
3,oagyyo,was just trying to help the driver.,orders $10 in food. tries to leave $5.50 tip. ...,more that?,maybe it prevents accidentally doing the wrong...,0
4,faw3bs,wow,"holy shit, i've seen a lot of these but that i...",the really boring dorian grey,step 1: dress up as subject.\n\nstep 2: walk u...,1
5,j905fx,someone put a paper trump sticker on my sister...,NaN,NaN,that’s so stupid. i don’t care if you’re a tru...,0
6,fvma5x,the dark side of the force is a pathway to man...,you refer to the prophecy of the one who will ...,this is why the police use slug-throwers,*deflects cop-force lightning*,1
7,1d86j7g,bros transformation is crazy,NaN,NaN,is this for real?,0
8,1dk04ox,miss deep dish,"a wise man once said\n\n""even a 747 looks smal...",internet inches lol,9 inches is 0.134 tom cruises,1
9,7ver8l,man ruthlessly assassinated by corporate machine,no sub is safe today huh,"this was cute, i like it.","nice hands, feet!",1


### Initializing new columns for llm generated annotations

In [8]:
df['ts_1'] = None
df['ts_2'] = None
df['ts_3'] = None
df['ts_4'] = None
df['ts_5'] = None
df.head()

,submissionId,submissionTitle,context1,context2,utterance,isTextSarcastic?,ts_1,ts_2,ts_3,ts_4,ts_5
0,1dncran,the clouds did a thing this morning...,looks like someone laid out some rails of blow,all the deities got together are they are havi...,colombian clouds,1,None,None,None,None,None
1,c9tdn7,looks like i won't be listening to my new viny...,it’s even crammed in there so hard your mailbo...,is there's a way to ask for a refund from the ...,how can an assigned delivery person not know t...,0,None,None,None,None,None
2,kctb6e,sounded way better in my head,i hope we get our 9-month cupcake in a cup.,bruh wall-e's my fav pixar movie! take my upvo...,i remember when it was just supposed to be a m...,0,None,None,None,None,None
3,oagyyo,was just trying to help the driver.,orders $10 in food. tries to leave $5.50 tip. ...,more that?,maybe it prevents accidentally doing the wrong...,0,None,None,None,None,None
4,faw3bs,wow,"holy shit, i've seen a lot of these but that i...",the really boring dorian grey,step 1: dress up as subject.\n\nstep 2: walk u...,1,None,None,None,None,None


In [9]:
df.shape

(920, 11)

In [162]:
def generate_response(prompt: str):
    import requests
    import json

    url = "http://localhost:11434/api/generate"
    headers = {
        "Content-type": "applications/json"
    }

    data = {
        "model": "gemma2",
        "prompt": prompt,
        "stream": False,
        "format": "json",
        "options": {
            "temperature": 1
        }
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        response_text = response.text
        data = json.loads(response_text)
        return data['response']
    else:
        return ("Error:", response.status_code, response.text)


In [163]:
def provide_prompt_for_text_labels(context, utterance):
    message = """
    TASK:
    You will be given a context and an utterance, and your task is to determine how sarcastic the given utterance is.
    Rate each utterance on a scale of 0 to 10 for sarcasm, with 0 being completely non sarcastic and 10 being extremely sarcastic.

    To give you and idea:
    - Sarcasm is a subtle form of language in which people express the opposite of what is implied.
    - The utterance must imply a strong sarcastic irony, insult, or juxtaposition when considered together with the context.
    - If the utterance fails to do so , the utterance is non sarcastic by default.
    - Most utterances does not imply any sarcasm at all.
    
    Given the following few shot examples (delimited by <examples></examples>)
    <examples>
    Context: so mr thompson, you say you have some programming skills???
    Utterance: probably referring to the "readability" certification. it's not hard, it just exempts you from needing a peer review on commits and enables you to approve other people's commits. totally overblown. i had readability in go, python, golang, and c when i worked there. you use an existing commit to request a readability certification.
    Your Response: {"Rating": "0.5"}
    ----

    Context: we’ve come full circle
    Utterance: there are three things you don’t discuss with strangers: religion, politics, and star wars.
    Your Response: {"Rating": "9.5"}
    ----

    Context: "clone wars? what are you talking about?"
    Utterance: i am one of the guys that saw star wars as a little boy (age 7, saw it 1978) the line was perfect!it immediately triggered my imagination of a dystopia where humans were cloned for being soldiers and maybe rebelled or tried to dominate which lead to a war. not far away from what was created later.
    Your Response: {"Rating": "0.7"}
    ----

    Context: keepers of peace, but in the most violent way
    Utterance: cut their hand off. that's the sith way. or go full obi and cut both their arms and legs off. that'll show em. bastards.
    Your Response: {"Rating": "8.7"}
    ----

    Context: people dont like videos cut too short.
    Utterance: then later, they would cut off just one wire from a 1,000,000 volt power grid
    Your Response: {"Rating": "2.3"}
    ----

    Context: roommates rice cooker has turned into brain-like tissue. he refuses to clean it, and leaves it on the shared kitchen counter.
    Utterance: that’s a biohazard.
    Your Response: {"Rating": "8.3"}
    ----

    Context: found a camera in my air b&b. it was halfway behind the painting with only the lens peeking out. discovered it because we heard it clicking after me and my girl got out of the shower.
    Utterance: that's not mildly infuriating, that's much worse.
    Your Response: {"Rating": "0.6"}
    ----

    Context: sick fuck fuses dogs togeter with black sorcery
    Utterance: i'm so confused rn
    Your Response: {"Rating": "8.4"}
    ----

    Context: this restaurant covered up the "no tip" option with a sticker to force tipping
    Utterance: percentage = 0%
    Your Response: {"Rating": "2.4"}
    ----

    Context: looks like i won't be listening to my new vinyl record. thanks, usps
    Utterance: how can an assigned delivery person not know the basic rule of don't bend the damn packages?!
    Your Response: {"Rating": "1.5"}
    </examples>
    ----
    Context: someone put a paper trump sticker on my sister’s car because we have a biden sign in our front yard.
    Utterance: that’s so stupid. i don’t care if you’re a trump supporter or a biden supporter, do not vandalize other peoples property because they like a different candidate then you.
    Your Response: {"Rating": "0.6"}
    """

    message = message + f"""
    Generate a valid JSON for the following(delimited by <test></test>)
    <test>
    Context: {context}
    Utterance: {utterance}
    Your Response: 
    </test>
    Respond only with valid JSON. Do not write an introduction or summary.
    """
    
    return message

def detect_sarcasm_in_text(context, utterance) -> str:
    response = generate_response(
        provide_prompt_for_text_labels(context, utterance)
    )
    return response

In [164]:
detect_sarcasm_in_text("My name is Sinngam", "Hi sinngam")

'{"Rating": "0"}'

In [149]:
detect_sarcasm_in_text(
    "cia agent waterboards suspected al-qaeda leader (2003)", 
    "this is a piece of art.")

'{"Rating": "9.8"}  \n'

In [150]:
detect_sarcasm_in_text(
    "me after surviving an accidental perma ban",
    "meesa propose that the senate give immediately emergency powers to /u/sexygungan69", )

'{"Rating": "7.8"}'

## Label Generation

In [165]:
curIndex = 0

In [166]:
curIndex

0

# Use for Rating style prompt

In [170]:
for i in range(curIndex, df.shape[0]):
    curIndex = i
    try:
        response = detect_sarcasm_in_text(df.iloc[i]['submissionTitle'], df.iloc[i]['utterance'])
        response_as_dict = json.loads(response)
        response_rating = float(response_as_dict['Rating'])
    except Exception as e:
        print(e)
        response_rating = 0
    is_sarcastic = int(response_rating > 5)
    print(df.iloc[i]['utterance'])

    print("Response Rating: ", response_rating, " Original: ", df.iloc[i]['isTextSarcastic?'], "New: ", is_sarcastic)
    df.at[i, 'ts_8'] = is_sarcastic

colombian clouds
Response Rating:  0.2  Original:  1 New:  0
how can an assigned delivery person not know the basic rule of don't bend the damn packages?!
Response Rating:  1.5  Original:  0 New:  0
i remember when it was just supposed to be a march break extension, damn
Response Rating:  1.9  Original:  0 New:  0
maybe it prevents accidentally doing the wrong amount but in that case it should ask you “are you sure?” instead of not letting you. kinda messed up
Response Rating:  3.2  Original:  0 New:  0
step 1: dress up as subject.

step 2: walk up to people looking at the portrait.

step 3: ???

step 4: profit.
Response Rating:  8.0  Original:  1 New:  1
that’s so stupid. i don’t care if you’re a trump supporter or a biden supporter, do not vandalize other peoples property because they like a different candidate then you.
Response Rating:  0.6  Original:  0 New:  0
*deflects cop-force lightning*
Response Rating:  9.2  Original:  1 New:  1
is this for real?
Response Rating:  0.1  Origi

# Use for discrete output prompt (0 or 1)

In [142]:
for i in range(curIndex, df.shape[0]):
    curIndex = i
    try:
        response = detect_sarcasm_in_text(df.iloc[i]['submissionTitle'], df.iloc[i]['utterance'])
        response_as_dict = json.loads(response)
        response_rating = int(response_as_dict['Rating'])
    except Exception as e:
        print(e)
        response_rating = 0
    is_sarcastic = response_rating
    print(df.iloc[i]['utterance'])

    print(" Original: ", df.iloc[i]['isTextSarcastic?'], "New: ", is_sarcastic)
    df.at[i, 'ts_5'] = is_sarcastic

colombian clouds
 Original:  1 New:  1
how can an assigned delivery person not know the basic rule of don't bend the damn packages?!
 Original:  0 New:  0
i remember when it was just supposed to be a march break extension, damn
 Original:  0 New:  1
maybe it prevents accidentally doing the wrong amount but in that case it should ask you “are you sure?” instead of not letting you. kinda messed up
 Original:  0 New:  1
step 1: dress up as subject.

step 2: walk up to people looking at the portrait.

step 3: ???

step 4: profit.
 Original:  1 New:  1
that’s so stupid. i don’t care if you’re a trump supporter or a biden supporter, do not vandalize other peoples property because they like a different candidate then you.
 Original:  0 New:  0
*deflects cop-force lightning*
 Original:  1 New:  1
is this for real?
 Original:  0 New:  0
9 inches is 0.134 tom cruises
 Original:  1 New:  1
nice hands, feet!
 Original:  1 New:  1
isn't this from a panera bread?
 Original:  0 New:  1
what’s wrong w

In [169]:
df.head()

,submissionId,submissionTitle,context1,context2,utterance,isTextSarcastic?,ts_1,ts_2,ts_3,ts_4,ts_5,ts_6,ts_7,ts_8
0,1dncran,the clouds did a thing this morning...,looks like someone laid out some rails of blow,all the deities got together are they are havi...,colombian clouds,1,0,0,1,0,1,0.0,0.0,None
1,c9tdn7,looks like i won't be listening to my new viny...,it’s even crammed in there so hard your mailbo...,is there's a way to ask for a refund from the ...,how can an assigned delivery person not know t...,0,0,0,0,0,0,0.0,0.0,None
2,kctb6e,sounded way better in my head,i hope we get our 9-month cupcake in a cup.,bruh wall-e's my fav pixar movie! take my upvo...,i remember when it was just supposed to be a m...,0,0,1,1,0,1,0.0,0.0,None
3,oagyyo,was just trying to help the driver.,orders $10 in food. tries to leave $5.50 tip. ...,more that?,maybe it prevents accidentally doing the wrong...,0,0,1,0,0,1,0.0,0.0,None
4,faw3bs,wow,"holy shit, i've seen a lot of these but that i...",the really boring dorian grey,step 1: dress up as subject.\n\nstep 2: walk u...,1,1,1,1,1,1,1.0,1.0,None


In [40]:
df['isTextSarcastic?'].value_counts()

isTextSarcastic?
1    534
0    386
Name: count, dtype: int64

In [171]:
df['ts_8'].value_counts()

ts_8
1    621
0    299
Name: count, dtype: int64

# Evaluation

In [155]:
def compute_accuracy(y_true, y_pred):
    """
    Computes accuracy by comparing the true labels with the predicted labels.
    """
    correct_predictions = sum(y_t == y_p for y_t, y_p in zip(y_true, y_pred))
    accuracy = correct_predictions / len(y_true)
    return accuracy

def compute_precision(y_true, y_pred):
    """
    Computes precision by calculating the ratio of true positives to the sum of true and false positives.
    """
    true_positives = sum((y_t == 1 and y_p == 1) for y_t, y_p in zip(y_true, y_pred))
    predicted_positives = sum(y_p == 1 for y_p in y_pred)
    precision = true_positives / predicted_positives if predicted_positives > 0 else 0
    return precision

def compute_recall(y_true, y_pred):
    """
    Computes recall by calculating the ratio of true positives to the sum of true positives and false negatives.
    """
    true_positives = sum((y_t == 1 and y_p == 1) for y_t, y_p in zip(y_true, y_pred))
    actual_positives = sum(y_t == 1 for y_t in y_true)
    recall = true_positives / actual_positives if actual_positives > 0 else 0
    return recall

def compute_f1(precision, recall):
    """
    Computes F1 score as the harmonic mean of precision and recall.
    """
    if precision + recall == 0:
        return 0
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

def evaluate(y_true, y_pred):
    accuracy = compute_accuracy(y_true, y_pred)
    precision = compute_precision(y_true, y_pred)
    recall = compute_recall(y_true, y_pred)
    f1 = compute_f1(precision, recall)
    print("\tAccuracy: ", accuracy, "\n", "\tPrecision: ", precision,"\n", "\tRecall: ", recall, "\n", "\tF1 score: ", f1)
    

In [172]:
evaluate(df["isTextSarcastic?"].to_list(), df["ts_8"].to_list())

	Accuracy:  0.7206521739130435 
 	Precision:  0.7230273752012882 
 	Recall:  0.8408239700374532 
 	F1 score:  0.7774891774891776


In [173]:
df.to_csv('labelled_csv/gemma2-sinngam.csv')